# Modelagem Matemática de Doenças Infecciosas (Estudos)

## Atividade (11/03/24): Ajuste de dados de incidência de COVID-19 em São Paulo ao modelo SIR

**Bibliotecas utilizadas:**

* numpy: Importa a biblioteca NumPy com um alias np, que é comumente usado para operações numéricas.
* pandas: Importa a biblioteca Pandas com um alias pd, que é usada para manipulação e análise de dados.
* odeint: Importa a função odeint da subbiblioteca integrate da biblioteca SciPy, que é usada para integrar equações diferenciais ordinárias.
* minimize: Importa a função minimize da sub-biblioteca optimize da biblioteca SciPy, que é usada para minimizar funções.
* matplotlib.pyplot: Importa a biblioteca Matplotlib com um alias plt, que é usada para criar gráficos e visualizações.
* plotly.io: importa a biblioteca de visualização de dados interativa e open-source em Python. Ela permite criar gráficos interativos de alta qualidade para análise de dados e comunicação visual. 

**Procedimntos**

1. **Definição das equações diferenciais ordinárias:** Em nossas análises consideramos o modelo SIR, dado por

$$\left\{\begin{array}{llll}
         \dfrac{dS}{dt}=-\beta SI,\\
         \\
         \dfrac{dI}{dt}=\beta SI - \gamma I,\\
         \\
         \dfrac{dR}{dt}=\gamma I,
\end{array}\right.$$

em que $\beta SI$ é a força de infecção (lembrando que estamos considerando uma população constante), e $\gamma I$ o termo de recuperação ou morte de indivíduos infectados.

1. **Definição da função custo:** Visa quantificar o quão bem o modelo SIR se ajusta aos dados reais de casos confirmados de COVID-19. Os parâmetros a serem otimizados pelo método dos mínimos quadrados, como a taxa de transmissão ($\beta$) e a taxa de recuperação ($\gamma$), são passados como entrada para a função. Primeiro, são definidas as condições iniciais do modelo, incluindo o número inicial de indivíduos infectados e recuperados, com base nos dados reais. Em seguida, as equações diferenciais do modelo SIR são resolvidas numericamente para estimar o número de casos confirmados ao longo do tempo. O erro quadrático médio entre os casos simulados e os casos reais é calculado e retornado como a função de custo. Minimizar esse erro é essencial para encontrar os melhores valores dos parâmetros do modelo que se ajustem aos dados observados, fornecendo assim uma representação precisa da propagação da doença.

2. **Método de otimização para o ajuste dos mínimos quadrados:**  O método L-BFGS-B (*Limited-memory Broyden-Fletcher-Goldfarb-Shanno with Bound Constraints*) é um método de otimização que combina elementos do método BFGS (Broyden-Fletcher-Goldfarb-Shanno) com técnicas de limitação de memória. O método L-BFGS-B é adequado para problemas de ajuste por mínimos quadrados, especialmente quando a função objetivo é suave e as restrições de limite podem ser aplicadas aos parâmetros. Quando se trata de ajuste de mínimos quadrados, o objetivo é encontrar os parâmetros do modelo que minimizam a soma dos quadrados das diferenças entre os valores observados e os valores simulados pelo modelo. Algumas características do método:

    **a)** Eficiência: O L-BFGS-B é geralmente eficiente em encontrar o mínimo da função objetivo, especialmente para problemas de otimização suaves e bem-condicionados.

    **b)** Restrições de limite: Muitas vezes, é necessário impor limites nos parâmetros do modelo durante o ajuste. O L-BFGS-B suporta restrições de limite, o que significa que você pode especificar limites superiores e inferiores para os parâmetros.

    **c)** Robustez: O L-BFGS-B é relativamente robusto e pode lidar com uma variedade de problemas de ajuste, desde que as condições de viabilidade sejam satisfeitas.

O algoritmo a seguir consiste no ajuste pelo método dos mínimos quadrados do modelo SIR a um conjunto de dados de incidência de COVID-19 em São Paulo no período de 15/03/2020 a 03/09/2020.

In [23]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio

# Função que retorna as equações diferenciais do modelo SIR
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

# Função de custo para o método dos mínimos quadrados
def cost_function(params, t, data): # Esta linha define a função cost_function, que aceita três argumentos: params, t e data.  params é uma lista que contém os parâmetros a serem otimizados pelo método dos mínimos quadrados, neste caso, params inclui os parâmetros beta e gamma.  t representa a variável independente, neste caso, o tempo.  data contém os dados reais que queremos ajustar.
    N = 10000  # População total (apenas para normalização)
    beta, gamma = params # Extrai os parâmetros beta e gamma do vetor params. Esses parâmetros são as taxas de transmissão e de recuperação, respectivamente, do modelo SIR.
    
    # Condições iniciais
    I0 = data[0] # Define o número inicial de indivíduos infectados (I0) como o primeiro valor dos dados reais 
    R0 = 0 # Define o número inicial de indivíduos recuperados (R0) como zero. Isso porque o modelo SIR começa com todos os indivíduos suscetíveis, nenhum recuperado e alguns infectados.
    S0 = N - I0 - R0 # Calcula o número inicial de indivíduos suscetíveis (S0) subtraindo o número inicial de infectados (I0) e recuperados (R0) da população total (N).
    y0 = S0, I0, R0 # Define as condições iniciais do sistema de equações diferenciais, representadas por y0. Consiste em uma tupla contendo o número inicial de indivíduos suscetíveis, infectados e recuperados.
    
    # Integração das equações diferenciais
    solution = odeint(deriv, y0, t, args=(N, beta, gamma))
    
    # Apenas comparamos os casos confirmados (I) com os dados reais
    # Extrai a coluna correspondente aos casos confirmados (infectados) da solução do modelo SIR.
    # No modelo SIR, a segunda coluna da solução (solution[:, 1]) representa o número de indivíduos infectados.
    simulated_cases = solution[:, 1]
    
    # Calcula o erro quadrático médio entre os casos simulados e os casos reais.
    # A diferença entre os casos simulados e reais é elevada ao quadrado ((simulated_cases - data) ** 2), e então a média é calculada usando np.mean().
    # Esta média quadrática é o valor retornado pela função de custo. O objetivo é minimizar esse valor, ajustando os parâmetros beta e gamma. Quanto menor o erro, melhor o ajuste dos parâmetros aos dados reais.
    return np.mean((simulated_cases - data) ** 2)

# Carregando os dados do arquivo de dados
# Esta linha carrega os dados do arquivo "covid-sp.dat" para um DataFrame do pandas chamado data.
# O parâmetro sep="\s+" indica que os dados são separados por espaços em branco.
# header=None especifica que o arquivo não contém linha de cabeçalho.
# skiprows=1 pula a primeira linha do arquivo, que não contém dados.
# names=["Date", "Cases"] atribui nomes às colunas do DataFrame, sendo "Date" para as datas e "Cases" para o número de casos.
data = pd.read_csv("covid-sp.dat", sep="\s+", header=None, skiprows=1, names=["Date", "Cases"])

# Converter a coluna de datas para o formato datetime
# Esta linha converte a coluna 'Date' do DataFrame para o formato datetime do pandas.
# Isso é feito usando a função pd.to_datetime(), que converte valores de data e hora para o formato datetime.
data['Date'] = pd.to_datetime(data['Date'])

# Calcular o número de dias a partir da primeira data
# Esta linha calcula o número de dias a partir da primeira data presente nos dados.
# data['Date'].iloc[0] retorna a primeira data no DataFrame.
# data['Date'] - data['Date'].iloc[0] calcula a diferença entre cada data no DataFrame e a primeira data.
# .dt.days extrai apenas o número de dias dessa diferença.
# O resultado é atribuído à nova coluna 'Days' no DataFrame, que representa os dias decorridos desde o início dos dados.
data['Days'] = (data['Date'] - data['Date'].iloc[0]).dt.days

# Estimativa inicial para os parâmetros beta e gamma. Esses valores são usados como ponto de partida para o processo de otimização.
initial_guess = [0.001, 0.1]

# Executar o ajuste dos dados ao modelo SIR
# A função minimize é usada para minimizar a função de custo definida anteriormente (cost_function).
# cost_function: A função de custo que calcula o erro entre os dados reais e os simulados.
# initial_guess: Estimativa inicial dos parâmetros a serem otimizados.
# args=(data['Days'], data['Cases']): Argumentos adicionais passados para a função de custo, neste caso, os dias decorridos e o número de casos confirmados.
# method='L-BFGS-B': O método de otimização a ser usado. Neste caso, o método de otimização limitada de BFGS (L-BFGS-B) é escolhido. Este é um método de otimização sem derivadas, adequado para problemas de otimização não suaves.
result = minimize(cost_function, initial_guess, args=(data['Days'], data['Cases']), method='L-BFGS-B')

# Extrair os parâmetros ajustados
beta, gamma = result.x

# Condições iniciais e integração das equações diferenciais com os parâmetros ajustados
N = 10000
I0 = data['Cases'].iloc[0]
R0 = 0
S0 = N - I0 - R0
y0 = S0, I0, R0
solution = odeint(deriv, y0, data['Days'], args=(N, beta, gamma))
S, I, R = solution.T

# Visualizar os resultados
# Criar as linhas de dados para o gráfico
real_data = go.Scatter(x=data['Date'], y=data['Cases'], mode='markers', name='Dados Reais (I)')
model_data = go.Scatter(x=data['Date'], y=I, mode='lines', name='Modelo SIR (I)')

# Criar o layout do gráfico
layout = go.Layout(
    title='Ajuste dos Dados de COVID-19 ao Modelo SIR',
    xaxis=dict(title='Data'),
    yaxis=dict(title='Casos Confirmados (I)'),
    hovermode='x'  # Ativar o modo de interação hover para mostrar os valores diretamente na curva
)

# Criar a figura do gráfico
fig = go.Figure(data=[real_data, model_data], layout=layout)

# Salvar
pio.write_html(fig, 'Ajuste_SIR.html')

# Exibir o gráfico interativo
fig.show()

# Exibir os parâmetros ajustados
print("Parâmetros Ajustados:")
print("Beta:", beta)
print("Gamma:", gamma)

# Calcular o Número Reprodutivo Básico (R0)
R0 = beta / gamma
print("Número Reprodutivo Básico (R0):", R0)




Parâmetros Ajustados:
Beta: 0.07769302663053514
Gamma: 0.05698903758703152
Número Reprodutivo Básico (R0): 1.3632977484816313


<iframe src="Atividade_1/Ajuste_SIR.html" width="1000" height="600"></iframe>


Precisamos conferir se o ajuste do modelo SIR aos dados é adequado e, para isso, fizemos uso do Índice de Correlação Intraclasse (ICC) para medir a consistência do ajuste.

### Índice de correlação Intraclasse

O Índice de Correlação Intraclasse (ICC) é uma medida estatística que avalia a consistência ou concordância entre as medidas repetidas em uma mesma amostra. A categorização dos valores de ICC baseia-se nas diretrizes apresentadas por Cicchetti (1994) e são dadas da seguinte forma: "Inaceitável" designa valores de ICC abaixo de 0,7; "Fraco" corresponde a valores de ICC que variam de 0,7 a 0,79; "Bom" abrange valores de ICC entre 0,8 e 0,89; e "Excelente" denota valores de ICC que se enquadram no intervalo de 0,90 a 1. No contexto do ajuste de dados ao modelo SIR, podemos calcular o ICC para avaliar a qualidade do ajuste comparando os casos reais com os casos simulados pelo modelo. Para calcular o ICC, primeiro precisamos calcular a soma total dos quadrados (SST), que é a variação total dos dados. Em seguida, calculamos a soma dos quadrados dos resíduos (SSE), que é a variação não explicada pelo modelo. O ICC é então dado pela razão entre a diferença entre SST e SSE e SST.

O algoritmo para o cálculo do ICC é dado a seguir:

In [24]:
# Calcular a Soma Total dos Quadrados (SST)
SST = np.sum((data['Cases'] - np.mean(data['Cases'])) ** 2)

# Calcular a Soma dos Quadrados dos Resíduos (SSE)
SSE = np.sum((data['Cases'] - I) ** 2)

# Calcular o Índice de Correlação Intraclasse (ICC)
ICC = (SST - SSE) / SST

print("Índice de Correlação Intraclasse (ICC):", ICC)

Índice de Correlação Intraclasse (ICC): 0.3753578998204466


Os resultados obtidos mostram que, para o conjunto de dados disponibilizados, a taxa de infecção, $\beta$, foi estimada em aproximadamente 0,776 e, a taxa de recuperação, $\gamma$ foi estimada em aproximadamente 0,056. Além disso, foi possível fazer uma estimativa do Número Reprodutivo Básico,  $\mathcal{R}_0$, com um valor de aproximadamente 1,36. De acordo com o ICC obtido, de aproximadamente 0,37, o ajuste do modelo SIR aos dados não é adequado e considerado inadequado.

**Discussão em sala de aula:** Os resultados indicam uma certa disparidade com o que foi relatado no período da pandemia, no que diz respeito ao número reprodutivo básico, bem como o período infeccioso, calculado em  aproximadamente 17 dias. Isso é explicado pelo fato de que, para o ajuste do modelo, utilizei todo o conjunto de dados, sendo que, seria necessário a utilização de apenas uma parte, a saber, o período  de crescimento da doença, que se aproxima de uma reta, havendo a possibilidade de extrair informações mais precisas. Podemos notar que o ajuste se aproxima de uma reta até o início de maio, logo, podemos calcular a inclinação desta reta para determinar o $\mathcal{R}_0$ mais aproximado. 

In [25]:
# Filtrar os dados até aproximadamente o mês de maio (considerando até o dia 31 de maio)
data_until_may = data[data['Date'] <= '2020-05-31']

# Ajustar uma reta aos dados reais usando o método dos mínimos quadrados
x = data_until_may['Days']  # Dados de tempo (dias)
y = data_until_may['Cases']  # Dados de casos confirmados (reais)
slope, _ = np.polyfit(x, y, 1)  # Ajustar uma reta de primeira ordem (y = mx + c) aos dados usando o método dos mínimos quadrados e extrair a inclinação (coeficiente angular) da reta

# Exibir a inclinação da reta crescente
print("Inclinação da reta crescente até maio:", slope)

Inclinação da reta crescente até maio: 3.163886746165228


O coeficiente angular da reta ajustada aos dados, que representa $\beta$, foi de aproximadamente 3,16 e, calculando o $\mathcal{R}_0$ a partir deste resultado temos um valor aproximado de 56,42, dado que o $\gamma$ estimado foi de 0,056.

### Número Reprodutivo Efetivo $\mathcal{R}_t$

**Definição:** É uma medida que representa o número médio de casos secundários gerados por um único caso em um determinado momento no tempo $t$, levando em consideração a dinâmica da epidemia. Definido para qualquer tempo $t$, após a invasão a fração de suscetíveis na população é menor do que 1, de maneira  que nem todos os contatos resultam em um novo caso. 

Nestas análises foram calculados três possibilidades para o $\mathcal{R}_t$:

* $\mathcal{R}_t$ considerando o $\mathcal{R}_0$ estimado pelo modelo SIR, fazendo $\mathcal{R}_t=\mathcal{R_0}\times S(t)/N$, em que $S(t)$ é a solução numérica do modelo ajustado aos dados;
  
* $\mathcal{R}_t$ considerando o coeficiente angular da reta ajustada ao início do período de infecção, que representa o parâmetro $\beta$ no modelo, fazendo $\mathcal{R}_t=\mathcal{R_0}\times S(t)/N$;

* $\mathcal{R}_t$ considerando a equação $\mathcal{R}_t = \dfrac{b(t)}{\int_0^\infty b(t-a)g(a)da}$, em que $b(t)$ representa o conjunto de dados, $b(t-a)$ o delay, e $g(a)=ae^{-at}$.

Os resultados são mostrados a seguir:

In [26]:
# Calcular o Número Reprodutivo Efetivo (Rt)
S_t = solution[:, 0]  # Número de indivíduos suscetíveis no tempo t
N = 10000  # População total

# Ajustado pelo coeficiente angula da reta no início da infecção
R0_angular = slope/gamma 
Rt_angular = R0_angular * (S_t / N)

# Ajustado pelo SIR
Rt=R0 * (S_t / N)

# Ajustado utilizando a integral
Rt_real = []
aux = 0
for i in data['Days']:    
    for j in range(data['Days'][0], i, 1):
        aux = aux + data['Cases'][i-j]*j*np.exp(-j*i)
        # print(aux)
    Rt_real.append(data['Cases'][i]/aux) 
Rt_real = Rt_real  * (S_t / N) 

C:\Users\guilh\AppData\Local\Temp\ipykernel_4852\520335057.py:19: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\guilh\AppData\Local\Temp\ipykernel_4852\520335057.py:19: RuntimeWarning:

divide by zero encountered in true_divide



In [27]:
# Criar o gráfico interativo de Rt
fig = go.Figure()

# Adicionar a linha de Rt
fig.add_trace(go.Scatter(x=data['Date'], y=Rt, mode='lines', name='Número Reprodutivo Efetivo (Rt)'))

# Adicionar a linha do limiar Rt=1
fig.add_shape(type='line', x0=data['Date'].iloc[0], y0=1, x1=data['Date'].iloc[-1], y1=1,
              line=dict(color='red', width=2, dash='dash'), name='Limiar (Rt=1)')

# Atualizar o layout do gráfico
fig.update_layout(title='Rt ajustado pelo modelo SIR',
                  xaxis_title='Data',
                  yaxis_title='Número Reprodutivo Efetivo (Rt)',
                  xaxis=dict(tickangle=-45),
                  showlegend=True,
                  legend=dict(x=0.02, y=0.95),
                  template='plotly_white')

# Salvar o gráfico em um arquivo HTML
pio.write_html(fig, 'Rt_SIR.html')

# Exibir o gráfico interativo
fig.show()


<iframe src="Atividade_1/Rt_SIR.html" width="1000" height="600"></iframe>


In [28]:
fig = go.Figure()
# Eliminamos os dois primeiros dias devido a divisão por zero associada a integração
fig.add_trace(go.Scatter(x=data['Date'][2:len(data['Date'])-1], y=Rt_real[2:len(data['Date'])-1], mode='markers', name='Número Reprodutivo Efetivo (Rt) a partir dos dados reais'))
fig.add_shape(type='line', x0=data['Date'].iloc[0], y0=1, x1=data['Date'].iloc[-1], y1=1,
              line=dict(color='red', width=2, dash='dash'), name='Limiar (Rt=1)')
fig.update_layout(title='Rt ajustado pela integração com os dados reais',
                  xaxis_title='Data',
                  yaxis_title='Número Reprodutivo Efetivo (Rt)',
                  xaxis=dict(tickangle=-45),
                  showlegend=True,
                  legend=dict(x=0.02, y=0.95),
                  template='plotly_white')

pio.write_html(fig, 'Rt_int_dados.html')

fig.show()

<iframe src="Atividade_1/Rt_int_dados.html" width="1000" height="600"></iframe>

In [29]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['Date'], y=Rt_angular, mode='lines', name='Rt ajustado pelo coef. angular'))

fig.add_shape(type='line', x0=data['Date'].iloc[0], y0=1, x1=data['Date'].iloc[-1], y1=1,
              line=dict(color='red', width=2, dash='dash'), name='Limiar (Rt=1)')

fig.update_layout(title='Rt ajustado pelo coeficiente angular da reta de crescimento do número de casos',
                  xaxis_title='Data',
                  yaxis_title='Número Reprodutivo Efetivo (Rt)',
                  xaxis=dict(tickangle=-45),
                  showlegend=True,
                  legend=dict(x=0.02, y=0.95),
                  template='plotly_white')

pio.write_html(fig, 'Rt_coef_ang.html')

fig.show()

<iframe src="Atividade_1/Rt_coef_ang.html" width="1000" height="600"></iframe>

### Testando o modelo SIS e SIRS aos dados

Como vimos, o modelo SIR não obteve um bom ajuste aos dados, logo, nesta seção verificaremos o ajuste a outros modelos epidemiológicos, a saber, SIS e SIRS.

* SIS: No modelo SIS, os indivíduos se recuperam da infecção e retornam à suscetibilidade, não havendo uma fase de recuperação permanente.
* No modelo SIRS, os indivíduos se recuperam da infecção, mas eventualmente perdem a imunidade e se tornam suscetíveis novamente. 

In [30]:
# Função que retorna as equações diferenciais do modelo SIS
def deriv_SIS(y, t, N, beta, gamma):
    S, I = y
    dSdt = -beta * S * I / N + gamma * I
    dIdt = beta * S * I / N - gamma * I
    return dSdt, dIdt

# Função que retorna as equações diferenciais do modelo SIRS
def deriv_SIRS(y, t, N, beta, gamma, delta):
    S, I, R = y
    dSdt = -beta * S * I / N + delta * R
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I - delta * R
    return dSdt, dIdt, dRdt

# Função que retorna as equações diferenciais do modelo SEIR
def deriv_SEIR(y, t, N, beta, gamma, delta, alpha):
    S, E, I, R = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - delta * E
    dIdt = delta * E - gamma * I
    dRdt = gamma * I
    return dSdt, dEdt, dIdt, dRdt

# Função que retorna as equações diferenciais do modelo SEIRS
def deriv_SEIRS(y, t, N, beta, gamma, delta, alpha, rho):
    S, E, I, R = y
    dSdt = -beta * S * I / N + rho * R
    dEdt = beta * S * I / N - delta * E
    dIdt = delta * E - gamma * I
    dRdt = gamma * I - rho * R
    return dSdt, dEdt, dIdt, dRdt

def deriv_SIRD(y, t, N, beta, gamma, delta):
    S, I, R, D = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I - delta * I
    dRdt = gamma * I
    dDdt = delta * I  # Equação diferencial para a variável D
    return dSdt, dIdt, dRdt, dDdt


# Função que retorna as equações diferenciais do modelo SEIRD
def deriv_SEIRD(y, t, N, beta, gamma, delta, alpha):
    S, E, I, R, D = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - delta * E
    dIdt = delta * E - gamma * I - alpha * I
    dRdt = gamma * I
    dDdt = alpha * I
    return dSdt, dEdt, dIdt, dRdt, dDdt

# Função de custo para o método dos mínimos quadrados para o modelo SIS
def cost_function_SIS(params, t, data):
    N = 10000  # População total (apenas para normalização)
    beta, gamma = params
    # Condições iniciais
    I0 = data[0]
    S0 = N - I0
    y0 = S0, I0
    # Integração das equações diferenciais
    solution = odeint(deriv_SIS, y0, t, args=(N, beta, gamma))
    # Apenas comparamos os casos confirmados (I) com os dados reais
    simulated_cases = solution[:, 1]
    return np.mean((simulated_cases - data) ** 2)

# Função de custo para o método dos mínimos quadrados para o modelo SIRS
def cost_function_SIRS(params, t, data):
    N = 10000  # População total (apenas para normalização)
    beta, gamma, delta = params
    # Condições iniciais
    I0 = data[0]
    R0 = 0
    S0 = N - I0 - R0
    y0 = S0, I0, R0
    # Integração das equações diferenciais
    solution = odeint(deriv_SIRS, y0, t, args=(N, beta, gamma, delta))
    # Apenas comparamos os casos confirmados (I) com os dados reais
    simulated_cases = solution[:, 1]
    return np.mean((simulated_cases - data) ** 2)

# Função de custo para o método dos mínimos quadrados para o modelo SEIR
def cost_function_SEIR(params, t, data):
    N = 10000  # População total (apenas para normalização)
    beta, gamma, delta, alpha = params
    # Condições iniciais
    I0 = data[0]
    E0 = I0 * 2  # Assumindo que o número de expostos é o dobro do número inicial de infectados
    R0 = 0
    S0 = N - I0 - E0 - R0
    y0 = S0, E0, I0, R0
    # Integração das equações diferenciais
    solution = odeint(deriv_SEIR, y0, t, args=(N, beta, gamma, delta, alpha))
    # Apenas comparamos os casos confirmados (I) com os dados reais
    simulated_cases = solution[:, 2]  # I é o terceiro elemento de y (índice 2)
    return np.mean((simulated_cases - data) ** 2)

# Função de custo para o método dos mínimos quadrados para o modelo SEIRS
def cost_function_SEIRS(params, t, data):
    N = 10000  # População total (apenas para normalização)
    beta, gamma, delta, alpha, rho = params
    # Condições iniciais
    I0 = data[0]
    E0 = I0 * 2  # Assumindo que o número de expostos é o dobro do número inicial de infectados
    R0 = 0
    S0 = N - I0 - E0 - R0
    y0 = S0, E0, I0, R0
    # Integração das equações diferenciais
    solution = odeint(deriv_SEIRS, y0, t, args=(N, beta, gamma, delta, alpha, rho))
    # Apenas comparamos os casos confirmados (I) com os dados reais
    simulated_cases = solution[:, 2]  # I é o terceiro elemento de y (índice 2)
    return np.mean((simulated_cases - data) ** 2)

# Função de custo para o método dos mínimos quadrados para o modelo SIRD
def cost_function_SIRD(params, t, data):
    N = 10000  # População total (apenas para normalização)
    beta, gamma, delta = params
    # Condições iniciais
    I0 = data[0]
    R0 = 0
    D0 = 0
    S0 = N - I0 - R0 - D0
    y0 = S0, I0, R0, D0
    # Integração das equações diferenciais
    solution = odeint(deriv_SIRD, y0, t, args=(N, beta, gamma, delta))
    # Apenas comparamos os casos confirmados (I) com os dados reais
    simulated_cases = solution[:, 1]  # I é o segundo elemento de y (índice 1)
    return np.mean((simulated_cases - data) ** 2)

# Função de custo para o método dos mínimos quadrados para o modelo SEIRD
def cost_function_SEIRD(params, t, data):
    N = 10000  # População total (apenas para normalização)
    beta, gamma, delta, alpha = params
    # Condições iniciais
    I0 = data[0]
    E0 = I0 * 2  # Assumindo que o número de expostos é o dobro do número inicial de infectados
    R0 = 0
    D0 = 0
    S0 = N - I0 - E0 - R0 - D0
    y0 = S0, E0, I0, R0, D0
    # Integração das equações diferenciais
    solution = odeint(deriv_SEIRD, y0, t, args=(N, beta, gamma, delta, alpha))
    # Apenas comparamos os casos confirmados (I) com os dados reais
    simulated_cases = solution[:, 1]  # I é o segundo elemento de y (índice 1)
    return np.mean((simulated_cases - data) ** 2)

# Estimativa inicial para os parâmetros beta e gamma (SIR)
initial_guess_SIR = [0.001, 0.1]

# Estimativa inicial para os parâmetros beta e gamma (SIS)
initial_guess_SIS = [0.001, 0.1]

# Estimativa inicial para os parâmetros beta, gamma e delta (SIRS)
initial_guess_SIRS = [0.001, 0.1, 0.05]

# Estimativa inicial para os parâmetros beta e gamma (SIR)
initial_guess_SEIR = [0.001, 0.1, 0.1, 0.5]

# Estimativa inicial para os parâmetros beta e gamma (SIS)
initial_guess_SEIRS = [0.3, 0.1, 0.1, 0.5, 0.05]

# Estimativa inicial para os parâmetros beta, gamma e delta (SIRS)
initial_guess_SIRD = [0.001, 0.1, 0.05]

# Estimativa inicial para os parâmetros beta, gamma e delta (SIRS)
initial_guess_SEIRD = [0.001, 0.1, 0.1, 0.5]

# Executar o ajuste dos dados ao modelo SIR
result_SIR = minimize(cost_function, initial_guess_SIR, args=(data['Days'], data['Cases']), method='L-BFGS-B')

# Executar o ajuste dos dados ao modelo SIS
result_SIS = minimize(cost_function_SIS, initial_guess_SIS, args=(data['Days'], data['Cases']), method='L-BFGS-B')

# Executar o ajuste dos dados ao modelo SIRS
result_SIRS = minimize(cost_function_SIRS, initial_guess_SIRS, args=(data['Days'], data['Cases']), method='L-BFGS-B')

# Executar o ajuste dos dados ao modelo SIR
result_SEIR = minimize(cost_function_SEIR, initial_guess_SEIR, args=(data['Days'], data['Cases']), method='L-BFGS-B')

# Executar o ajuste dos dados ao modelo SIS
result_SEIRS = minimize(cost_function_SEIRS, initial_guess_SEIRS, args=(data['Days'], data['Cases']), method='L-BFGS-B')

# Executar o ajuste dos dados ao modelo SIRS
result_SIRD = minimize(cost_function_SIRD, initial_guess_SIRD, args=(data['Days'], data['Cases']), method='L-BFGS-B')

# Executar o ajuste dos dados ao modelo SIRS
result_SEIRD = minimize(cost_function_SEIRD, initial_guess_SEIRD, args=(data['Days'], data['Cases']), method='L-BFGS-B')


# Extrair os parâmetros ajustados
beta_SIR, gamma_SIR = result_SIR.x
beta_SIS, gamma_SIS = result_SIS.x
beta_SIRS, gamma_SIRS, delta_SIRS = result_SIRS.x
beta_SEIR, gamma_SEIR, delta_SEIR, alpha_SEIR = result_SEIR.x
beta_SEIRS, gamma_SEIRS, delta_SEIRS, alpha_SEIRS, rho_SEIRS = result_SEIRS.x
beta_SIRD, gamma_SIRD, delta_SIRD = result_SIRD.x
beta_SEIRD, gamma_SEIRD, delta_SEIRD, alpha_SEIRD = result_SEIRD.x

# Condições iniciais e integração das equações diferenciais com os parâmetros ajustados (SIR)
N = 10000
I0_SIR = data['Cases'].iloc[0]
R0_SIR = 0
S0_SIR = N - I0_SIR - R0_SIR
y0_SIR = S0_SIR, I0_SIR, R0_SIR
solution_SIR = odeint(deriv, y0_SIR, data['Days'], args=(N, beta_SIR, gamma_SIR))
S_SIR, I_SIR, R_SIR = solution_SIR.T

# Condições iniciais e integração das equações diferenciais com os parâmetros ajustados (SIS)
I0_SIS = data['Cases'].iloc[0]
S0_SIS = N - I0_SIS
y0_SIS = S0_SIS, I0_SIS
solution_SIS = odeint(deriv_SIS, y0_SIS, data['Days'], args=(N, beta_SIS, gamma_SIS))
S_SIS, I_SIS = solution_SIS.T

# Condições iniciais e integração das equações diferenciais com os parâmetros ajustados (SIRS)
I0_SIRS = data['Cases'].iloc[0]
R0_SIRS = 0
S0_SIRS = N - I0_SIRS - R0_SIRS
y0_SIRS = S0_SIRS, I0_SIRS, R0_SIRS
solution_SIRS = odeint(deriv_SIRS, y0_SIRS, data['Days'], args=(N, beta_SIRS, gamma_SIRS, delta_SIRS))
S_SIRS, I_SIRS, R_SIRS = solution_SIRS.T

# Condições iniciais e integração das equações diferenciais com os parâmetros ajustados (SEIR)
E0_SEIR = data['Cases'].iloc[0]  # Número inicial de indivíduos expostos
I0_SEIR = 1  # Número inicial de indivíduos infectados
R0_SEIR = 0  # Número inicial de indivíduos recuperados
S0_SEIR = N - E0_SEIR - I0_SEIR - R0_SEIR  # Número inicial de indivíduos suscetíveis
y0_SEIR = S0_SEIR, E0_SEIR, I0_SEIR, R0_SEIR
solution_SEIR = odeint(deriv_SEIR, y0_SEIR, data['Days'], args=(N, beta_SEIR, gamma_SEIR, delta_SEIR, alpha_SEIR))
S_SEIR, E_SEIR, I_SEIR, R_SEIR = solution_SEIR.T

# Condições iniciais e integração das equações diferenciais com os parâmetros ajustados (SEIRS)
E0_SEIRS = data['Cases'].iloc[0]  # Número inicial de indivíduos expostos
I0_SEIRS = 1  # Número inicial de indivíduos infectados
R0_SEIRS = 0  # Número inicial de indivíduos recuperados
S0_SEIRS = N - E0_SEIRS - I0_SEIRS - R0_SEIRS  # Número inicial de indivíduos suscetíveis
y0_SEIRS = S0_SEIRS, E0_SEIRS, I0_SEIRS, R0_SEIRS
solution_SEIRS = odeint(deriv_SEIRS, y0_SEIRS, data['Days'], args=(N, beta_SEIRS, gamma_SEIRS, delta_SEIRS, alpha_SEIRS, rho_SEIRS))
S_SEIRS, E_SEIRS, I_SEIRS, R_SEIRS = solution_SEIRS.T

# Condições iniciais e integração das equações diferenciais com os parâmetros ajustados (SIRD)
I0_SIRD = data['Cases'].iloc[0]  # Número inicial de indivíduos infectados
R0_SIRD = 0  # Número inicial de indivíduos recuperados
D0_SIRD = 0  # Número inicial de indivíduos que morreram
S0_SIRD = N - I0_SIRD - R0_SIRD - D0_SIRD  # Número inicial de indivíduos suscetíveis
y0_SIRD = S0_SIRD, I0_SIRD, R0_SIRD, D0_SIRD
solution_SIRD = odeint(deriv_SIRD, y0_SIRD, data['Days'], args=(N, beta_SIRD, gamma_SIRD, delta_SIRD))
S_SIRD, I_SIRD, R_SIRD, D_SIRD = solution_SIRD.T

# Condições iniciais e integração das equações diferenciais com os parâmetros ajustados (SEIRD)
E0_SEIRD = data['Cases'].iloc[0]  # Número inicial de indivíduos expostos
I0_SEIRD = 1  # Número inicial de indivíduos infectados
R0_SEIRD = 0  # Número inicial de indivíduos recuperados
D0_SEIRD = 0  # Número inicial de indivíduos mortos
S0_SEIRD = N - E0_SEIRD - I0_SEIRD - R0_SEIRD - D0_SEIRD  # Número inicial de indivíduos suscetíveis
y0_SEIRD = S0_SEIRD, E0_SEIRD, I0_SEIRD, R0_SEIRD, D0_SEIRD
solution_SEIRD = odeint(deriv_SEIRD, y0_SEIRD, data['Days'], args=(N, beta_SEIRD, gamma_SEIRD, delta_SEIRD, alpha_SEIRD))
S_SEIRD, E_SEIRD, I_SEIRD, R_SEIRD, D_SEIRD = solution_SEIRD.T

# Função para calcular o ICC
def calculate_ICC(data, simulated_cases):
    # Calcular a Soma Total dos Quadrados (SST)
    SST = np.sum((data - np.mean(data)) ** 2)
    # Calcular a Soma dos Quadrados dos Resíduos (SSE)
    SSE = np.sum((data - simulated_cases) ** 2)
    # Calcular o ICC
    ICC = (SST - SSE) / SST
    return ICC

# Calcular o ICC para cada modelo
ICC_SIR = calculate_ICC(data['Cases'], I_SIR)
ICC_SIS = calculate_ICC(data['Cases'], I_SIS)
ICC_SIRS = calculate_ICC(data['Cases'], I_SIRS)
ICC_SEIR = calculate_ICC(data['Cases'], I_SEIR)
ICC_SEIRS = calculate_ICC(data['Cases'], I_SEIRS)
ICC_SIRD = calculate_ICC(data['Cases'], I_SIRD)
ICC_SEIRD = calculate_ICC(data['Cases'], I_SEIRD)

# Exibir os parâmetros ajustados (SIR)
print("Parâmetros Ajustados (SIR):")
print("Beta:", beta_SIR)
print("Gamma:", gamma_SIR)
# Calcular o Número Reprodutivo Básico (R0) (SIR)
R0_SIR = beta_SIR / gamma_SIR
print("Número Reprodutivo Básico (R0) (SIR):", R0_SIR)
print("ICC para o modelo SIR:", ICC_SIR)

# Exibir os parâmetros ajustados (SIS)
print("\nParâmetros Ajustados (SIS):")
print("Beta:", beta_SIS)
print("Gamma:", gamma_SIS)
# Calcular o número reprodutivo básico (R0) ajustado para o modelo SIS
R0_SIS = beta_SIS / gamma_SIS
print("Número Reprodutivo Básico (R0) para o modelo SIS:", R0_SIS)
print("ICC para o modelo SIS:", ICC_SIS)

# Exibir os parâmetros ajustados (SIRS)
print("\nParâmetros Ajustados (SIRS):")
print("Beta:", beta_SIRS)
print("Gamma:", gamma_SIRS)
print("Delta:", delta_SIRS)
# Calcular o número reprodutivo básico (R0) ajustado para o modelo SIRS
R0_SIRS = beta_SIRS / gamma_SIRS
print("Número Reprodutivo Básico (R0) para o modelo SIRS:", R0_SIRS)
print("ICC para o modelo SIRS:", ICC_SIRS)

# Exibir os parâmetros ajustados (SEIR)
print("\nParâmetros Ajustados (SEIR):")
print("Beta:", beta_SEIR)
print("Gamma:", gamma_SEIR)
print("Delta:", delta_SEIR)
print("Alpha:", alpha_SEIR)
# Calcular o número reprodutivo básico (R0) ajustado para o modelo SEIR
R0_SEIR = beta_SEIR / gamma_SEIR
print("Número Reprodutivo Básico (R0) para o modelo SEIR:", R0_SEIR)
print("ICC para o modelo SEIR:", ICC_SEIR)

# Exibir os parâmetros ajustados (SEIRS)
print("\nParâmetros Ajustados (SEIRS):")
print("Beta:", beta_SEIRS)
print("Gamma:", gamma_SEIRS)
print("Delta:", delta_SEIRS)
print("Alpha:", alpha_SEIRS)
# Calcular o número reprodutivo básico (R0) ajustado para o modelo SEIR
R0_SEIR = beta_SEIRS / gamma_SEIRS
print("Número Reprodutivo Básico (R0) para o modelo SEIRS:", R0_SEIRS)
print("ICC para o modelo SEIRS:", ICC_SEIRS)

# Exibir os parâmetros ajustados (SIRD)
print("\nParâmetros Ajustados (SIRD):")
print("Beta:", beta_SIRD)
print("Gamma:", gamma_SIRD)
print("Delta:", delta_SIRD)
# Calcular o número reprodutivo básico (R0) ajustado para o modelo SIRD
R0_SIRD = beta_SIRD / gamma_SIRD
print("Número Reprodutivo Básico (R0) para o modelo SIRD:", R0_SIRD)
print("ICC para o modelo SIRD:", ICC_SIRD)

# Exibir os parâmetros ajustados (SEIRD)
print("\nParâmetros Ajustados (SEIRD):")
print("Beta:", beta_SEIRD)
print("Gamma:", gamma_SEIRD)
print("Delta:", delta_SEIRD)
print("Alpha:", alpha_SEIRD)
# Calcular o número reprodutivo básico (R0) ajustado para o modelo SEIRD
R0_SEIRD = beta_SEIRD / gamma_SEIRD
print("Número Reprodutivo Básico (R0) para o modelo SEIRD:", R0_SEIRD)
print("ICC para o modelo SEIRD:", ICC_SEIRD)




c:\Users\guilh\anaconda3\lib\site-packages\scipy\integrate\_odepack_py.py:248: ODEintWarning:

Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.



Parâmetros Ajustados (SIR):
Beta: 0.07769302663053514
Gamma: 0.05698903758703152
Número Reprodutivo Básico (R0) (SIR): 1.3632977484816313
ICC para o modelo SIR: 0.3753578998204466

Parâmetros Ajustados (SIS):
Beta: 8.66177369606229
Gamma: 8.231980851112322
Número Reprodutivo Básico (R0) para o modelo SIS: 1.0522101366273093
ICC para o modelo SIS: 0.023314048421058595

Parâmetros Ajustados (SIRS):
Beta: 0.13492869748682296
Gamma: 0.10102487857450523
Delta: 0.018550116762535946
Número Reprodutivo Básico (R0) para o modelo SIRS: 1.3355987098496125
ICC para o modelo SIRS: 0.6101268424203515

Parâmetros Ajustados (SEIR):
Beta: 0.07013907927966456
Gamma: 0.05370136984676818
Delta: 0.07215461969842397
Alpha: 0.5
Número Reprodutivo Básico (R0) para o modelo SEIR: 1.3060947882670375
ICC para o modelo SEIR: -3.974165317924201

Parâmetros Ajustados (SEIRS):
Beta: 0.3
Gamma: 0.1
Delta: 0.1
Alpha: 0.5
Número Reprodutivo Básico (R0) para o modelo SEIRS: 0
ICC para o modelo SEIRS: -69.50043424645949


In [31]:
# Criar o gráfico interativo
import plotly.io as pio

# Criar o gráfico interativo
fig = go.Figure()

# Adicionar os dados reais
fig.add_trace(go.Scatter(x=data['Date'], y=data['Cases'], mode='markers', name='Dados Reais (I)'))

# Adicionar o modelo SIR
fig.add_trace(go.Scatter(x=data['Date'], y=I_SIR, mode='lines', name='Modelo SIR (I)', line=dict(color='blue')))

# Adicionar o modelo SIS
fig.add_trace(go.Scatter(x=data['Date'], y=I_SIS, mode='lines', name='Modelo SIS (I)', line=dict(color='green', dash='dash')))

# Adicionar o modelo SIRS
fig.add_trace(go.Scatter(x=data['Date'], y=I_SIRS, mode='lines', name='Modelo SIRS (I)', line=dict(color='red', dash='dot')))

# Adicionar o modelo SEIR
fig.add_trace(go.Scatter(x=data['Date'], y=I_SEIR, mode='lines', name='Modelo SEIR (I)', line=dict(color='violet', dash='dot')))

# Adicionar o modelo SIRS
fig.add_trace(go.Scatter(x=data['Date'], y=I_SIRD, mode='lines', name='Modelo SIRD (I)', line=dict(color='orange', dash='dot')))

# Atualizar o layout do gráfico
fig.update_layout(title='Comparação dos Modelos Epidemiológicos com os Dados Reais',
                  xaxis_title='Data',
                  yaxis_title='Casos Confirmados (I)',
                  xaxis=dict(tickangle=-45),
                  showlegend=True,
                  legend=dict(x=0.02, y=0.95),
                  template='plotly_white')

# Salvar o gráfico em um arquivo HTML
pio.write_html(fig, 'comparacao_modelos.html')

# Exibir o gráfico interativo
fig.show()


<iframe src="Atividade_1/comparacao_modelos.html" width="1000" height="600"></iframe>



Com os resultados obtidos, podemos concluir que, dentre os modelos escolhidos, o que melhor se ajusta aos dados é o SIRS, com  um ICC de aproximadamente 0,61. Para os critérios, este índice ainda é inaceitável, mas melhor do que o obtido no modelo SIR. Deste modo, vamos utilizá-lo para calcular o $\mathcal{R}_t$.



In [32]:
# Calcular o Número Reprodutivo Efetivo (Rt)
S_t_SIRS = solution_SIRS[:, 0]  # Número de indivíduos suscetíveis no tempo t
N = 10000  # População total

# Ajustado pelo SIRS
Rt_SIRS=R0_SIRS * (S_t_SIRS / N)

fig = go.Figure()
# Eliminamos os dois primeiros dias devido a divisão por zero associada a integração

# fig.add_trace(go.Scatter(x=data['Date'][2:len(data['Date'])-1], y=Rt_real[2:len(data['Date'])-1], mode='markers', name='Número Reprodutivo Efetivo (Rt) a partir dos dados reais'))
fig.add_trace(go.Scatter(x=data['Date'], y=Rt_SIRS, mode='lines', name='Número Reprodutivo Efetivo (Rt)'))


fig.add_shape(type='line', x0=data['Date'].iloc[0], y0=1, x1=data['Date'].iloc[-1], y1=1,
              line=dict(color='red', width=2, dash='dash'), name='Limiar (Rt=1)')
fig.update_layout(title='Rt ajustado pelo modelo SIRS',
                  xaxis_title='Data',
                  yaxis_title='Número Reprodutivo Efetivo (Rt)',
                  xaxis=dict(tickangle=-45),
                  showlegend=True,
                  legend=dict(x=0.02, y=0.95),
                  template='plotly_white')

pio.write_html(fig, 'Rt_SIRS.html')

fig.show()


<iframe src="Atividade_1/Rt_SIRS.html" width="1000" height="600"></iframe>

### Dados utilizados: COVID-19 em São Paulo em 2023

In [33]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Exibir os dados completos do DataFrame
print(data)

          Date  Cases  Days
0   2020-03-15    312     0
1   2020-03-16    398     1
2   2020-03-17    365     2
3   2020-03-18    399     3
4   2020-03-19    387     4
5   2020-03-20    596     5
6   2020-03-21    447     6
7   2020-03-22    447     7
8   2020-03-23    565     8
9   2020-03-24    514     9
10  2020-03-25    495    10
11  2020-03-26    493    11
12  2020-03-27    443    12
13  2020-03-28    449    13
14  2020-03-29    418    14
15  2020-03-30    454    15
16  2020-03-31    405    16
17  2020-04-01    589    17
18  2020-04-02    451    18
19  2020-04-03    444    19
20  2020-04-04    425    20
21  2020-04-05    417    21
22  2020-04-06    469    22
23  2020-04-07    432    23
24  2020-04-08    467    24
25  2020-04-09    459    25
26  2020-04-10    593    26
27  2020-04-11    427    27
28  2020-04-12    482    28
29  2020-04-13    500    29
30  2020-04-14    501    30
31  2020-04-15    609    31
32  2020-04-16    513    32
33  2020-04-17    570    33
34  2020-04-18    60